<a href="https://colab.research.google.com/github/vbloise3/Data_Science/blob/master/HypothesisandInference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from typing import Tuple 
import math